In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

In [14]:
firefox_binary_path = r"C:\Program Files\Mozilla Firefox\firefox.exe"

# Set Firefox options
options = Options()
options.binary_location = firefox_binary_path

# Specify the path to geckodriver using the Service class
service = Service(r"F:\amad-study\scraping-study\firefox\geckodriver.exe")
driver = webdriver.Firefox(service=service, options=options)

In [15]:
driver.get("https://remoteok.com/remote-jobs")

# Infinite scroll simulation
SCROLL_PAUSE_TIME = 10  # Pause time after each scroll

In [16]:
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)

    # Check if there's a "Load more" or if the page hasn't changed
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:  # Break if no new content is loaded
        break

In [17]:
# Parse the loaded content
soup = BeautifulSoup(driver.page_source, "html.parser")

# Extract job details
jobs = soup.find_all("tr", class_="job")
csv_data = []

for job in jobs:
    try:
        title = job.find("td", class_="company_and_position").find("h2").text.strip()
        company = job.find("td", class_="company_and_position").find("h3").text.strip()
        location = (
            job.find("td", class_="company_and_position")
            .find("div", class_="location")
            .text.strip()
        )
        tags = job.find("td", class_="tags").find_all("h3")
        tag_list = [tag.text.strip() for tag in tags]
        csv_data.append(
            {
                "job_title": title,
                "company": company,
                "location": location,
                "tags": ",".join(tag_list),
            }
        )
    except AttributeError:
        print("Error parsing a job. Skipping.")

# Save to CSV
df = pd.DataFrame(csv_data)
df.to_csv(
    r"F:\amad-study\scraping-study\task-4\remoteok_infinite_scroll_jobs.csv",
    index=False,
    encoding="utf-8",
)

In [18]:
# Close the browser
driver.quit()